# Pandas: CSV nuskaitymas, bazinė analizė ir eksportas į Excel

Šiame faile pateikiamas nuoseklus pavyzdys, kaip su `pandas`:

- nuskaityti CSV failą į `DataFrame`
- peržiūrėti struktūrą ir duomenų tipus
- patikrinti trūkstamas reikšmes ir duplikatus
- atlikti rūšiavimą, filtravimą ir paprastą grupavimą
- sukurti naujus stulpelius (modifikavimas)
- eksportuoti rezultatą į Excel

Komentarai pateikiami techniniu, aiškinamuoju stiliumi.


In [ ]:
import pandas as pd

# Rekomenduojama versija patikrinimui (nebūtina, bet naudinga)
pd.__version__


In [29]:
pwd

'c:\\Users\\User\\OneDrive\\Desktop\\BIT\\2025 11 12 DUOM\\V Modulis_Python_2025 11 12 DUOM\\BIT_PYTHON\\Paskaita_11\\dataframes'

## 1. CSV failo nuskaitymas

Dažnos klaidos ir kaip jų išvengti:

- **Windows kelio (path) sintaksė**: simbolis `\` Python eilutėse yra escape ženklas. Patikimiausia naudoti `/` arba `r"..."`.
- **Neteisinga darbo direktorija**: kai failas nerandamas, verta patikrinti `os.getcwd()` arba nurodyti pilną kelią.
- **Netinkamas skyriklis**: jei CSV naudoja `;`, reikės `sep=';'`.

Žemiau pateikiamas pavyzdys su `/` keliu. Jei failo kelias skiriasi, pakeičiama tik reikšmė kabutėse.


In [ ]:
# Pakeisti šį kelią į savo failo vietą
file_path = "C:\\Users\\User\\OneDrive\\Desktop\\BIT\\2025 11 12 DUOM\\V Modulis_Python_2025 11 12 DUOM\\BIT_PYTHON\\Python_P\\Pandas Course Resources\\retail\\transactions.csv"

#C:\Users\User\OneDrive\Desktop\BIT\2025 11 12 DUOM\V Modulis_Python_2025 11 12 DUOM\BIT_PYTHON\Python_P\Pandas Course Resources\retail\transactions.csv
transactions = pd.read_csv(file_path)

# Greitas patikrinimas
type(transactions), transactions.shape


## 2. DataFrame pagrindai

`head()` ir `tail()` padeda greitai suprasti:

- ar stulpelių pavadinimai teisingi
- ar reikšmės „atrodo realistiškos“
- ar nėra akivaizdžių formatavimo problemų


In [5]:
transactions.head(10)


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
5,2013-01-02,5,1903
6,2013-01-02,6,2143
7,2013-01-02,7,1874
8,2013-01-02,8,3250
9,2013-01-02,9,2940


In [6]:
transactions.tail(7)


,date,store_nbr,transactions
83481,2017-08-15,48,2722
83482,2017-08-15,49,2814
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932
83487,2017-08-15,54,802


## 3. Duomenų tipai ir bendra informacija

`info()` parodo:

- eilučių skaičių
- kiek stulpeliuose yra netrūkstamų reikšmių
- duomenų tipus
- apytikslį atminties panaudojimą

Gera praktika: **datas paversti į `datetime` tipą**, kad būtų patogu atlikti laiko analizę.


In [7]:
transactions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [8]:
# Data konvertavimas į datetime
# Dažna klaida: palikti datas kaip 'object' ir vėliau gauti keistas filtravimo / grupavimo klaidas.

transactions["date"] = pd.to_datetime(transactions["date"], errors="coerce")

transactions.dtypes


date            datetime64[ns]
store_nbr                int64
transactions             int64
dtype: object

## 4. Aprašomoji statistika

`describe()` pateikia skaitinių stulpelių santrauką:

- `mean` – vidurkis
- `std` – standartinis nuokrypis (svyravimai)
- `min`, `max` – reikšmių ribos
- `25%`, `50%`, `75%` – kvartiliai (pavyzdžiui, `50%` yra mediana)

Tai padeda pastebėti:
- galimus „pikus“ (labai dideles reikšmes)
- neįprastai mažas reikšmes
- didelius svyravimus tarp įrašų


In [10]:
transactions.describe().round().T


,count,mean,min,25%,50%,75%,max,std
date,83488,2015-05-20 16:07:40.866232064,2013-01-01 00:00:00,2014-03-27 00:00:00,2015-06-08 00:00:00,2016-07-14 06:00:00,2017-08-15 00:00:00,NaN
store_nbr,83488.0,27.0,1.0,13.0,27.0,40.0,54.0,16.0
transactions,83488.0,1695.0,5.0,1046.0,1393.0,2079.0,8359.0,963.0


## 5. Trūkstamos reikšmės (blank / missing values)

Net jei `info()` rodo, kad reikšmės netrūksta, po konvertavimo į `datetime` gali atsirasti `NaT` (nepavykusios datos).

Dažna klaida: atlikti grupavimą ar skaičiavimus nepatikrinus trūkstamų reikšmių.


In [ ]:
transactions.isna().sum()


date            0
store_nbr       0
transactions    0
dtype: int64

## 6. Duplikatai

Duplikatai gali atsirasti:

- kai tas pats failas importuojamas kelis kartus
- kai duomenys eksportuojami su pasikartojančiomis eilutėmis

Patikrinama, kiek yra pilnai vienodų eilučių.


In [12]:
transactions.duplicated().sum()


0

## 7. Pasiekimas ir šalinimas (accessing & dropping)

Dažnos klaidos:

- bandyti pasirinkti kelis stulpelius be dvigubų skliaustų (turi būti `df[[...]]`)
- naudoti `drop()` ir tikėtis, kad originalas pasikeis (be `inplace=True` ar be priskyrimo)

Žemiau pateikiami keli pavyzdžiai.


In [14]:
# Vieno stulpelio pasirinkimas (gaunama Series)
transactions["transactions"].head(10)


0     770
1    2111
2    2358
3    3487
4    1922
5    1903
6    2143
7    1874
8    3250
9    2940
Name: transactions, dtype: int64

In [15]:
# Kelių stulpelių pasirinkimas (gaunamas DataFrame)
transactions[["store_nbr", "transactions"]].head(10)


,store_nbr,transactions
0,25,770
1,1,2111
2,2,2358
3,3,3487
4,4,1922
5,5,1903
6,6,2143
7,7,1874
8,8,3250
9,9,2940


In [16]:
# Stulpelio pašalinimas (originalas nekeičiama, nes nėra priskyrimo)
transactions.drop(columns=["store_nbr"]).head()


,date,transactions
0,2013-01-01,770
1,2013-01-02,2111
2,2013-01-02,2358
3,2013-01-02,3487
4,2013-01-02,1922


## 8. Rūšiavimas ir filtravimas (sorting & filtering)

Dažnos klaidos:

- pamiršti `ascending=False`, kai reikia didžiausių reikšmių viršuje
- filtravimo sąlygose pamiršti skliaustus (pvz. `(cond1) & (cond2)`)
- naudoti `and` vietoje `&` (pandas filtravimui naudojamas `&`)

Žemiau pateikiami pavyzdžiai.


In [18]:
# Didžiausios transakcijos
transactions.sort_values(by="transactions", ascending=True).head(10)


,date,store_nbr,transactions
65479,2016-09-07,43,5
52392,2016-01-02,2,6
52428,2016-01-04,1,10
57950,2016-04-17,53,33
58003,2016-04-18,53,54
74018,2017-02-20,30,97
59594,2016-05-18,54,179
25817,2014-07-11,30,203
59557,2016-05-18,16,238
76259,2017-04-03,45,292


In [19]:
# Filtravimas: dienos, kai transakcijų daugiau nei 5000
high_volume = transactions[transactions["transactions"] > 5000]
high_volume.head()


,date,store_nbr,transactions
224,2013-01-06,46,5401
1464,2013-02-02,44,5166
1467,2013-02-02,47,5021
2798,2013-03-03,44,5065
2800,2013-03-03,46,5095


## 9. Stulpelių modifikavimas (modifying columns)

Dažnos klaidos:

- bandyti naudoti `.dt` metodus, kai data nėra `datetime`
- kurti naują stulpelį be aiškios taisyklės (vėliau sunku paaiškinti logiką)

Žemiau:
- sukuriamas savaitės dienos stulpelis
- sukuriamas paprastas aktyvumo lygio stulpelis pagal transakcijų kiekį


In [20]:
# Savaitės diena
transactions["weekday"] = transactions["date"].dt.day_name()

# Paprasta kategorija pagal transakcijų kiekį
transactions["activity_level"] = transactions["transactions"].apply(
    lambda x: "High" if x > 3000 else "Normal"
)

transactions.head()


,date,store_nbr,transactions,weekday,activity_level
0,2013-01-01,25,770,Tuesday,Normal
1,2013-01-02,1,2111,Wednesday,Normal
2,2013-01-02,2,2358,Wednesday,Normal
3,2013-01-02,3,3487,Wednesday,High
4,2013-01-02,4,1922,Wednesday,Normal


## 10. Pagrindinės įžvalgos (bazinė analizė)

Duomenyse yra:

- `date` – data
- `store_nbr` – parduotuvės numeris
- `transactions` – transakcijų skaičius (dienos aktyvumas)

Pateikiami keli paprasti pjūviai:

- vidutinis transakcijų skaičius pagal parduotuvę
- bendras transakcijų skaičius pagal datą
- top dienos pagal transakcijų skaičių


In [22]:
# Vidutinis transakcijų skaičius pagal parduotuvę (top 10)
avg_by_store = (
    transactions.groupby("store_nbr")["transactions"]
    .mean()
    .sort_values(ascending=False)
)

avg_by_store.tail(10)


store_nbr
10    986.562985
25    941.400619
13    938.248210
16    873.284436
54    865.924821
22    751.350224
30    707.631420
35    670.578162
32    635.197376
26    634.719309
Name: transactions, dtype: float64

In [23]:
# Bendras transakcijų skaičius pagal datą (pirmos 10 datų)
daily_total = (
    transactions.groupby("date")["transactions"]
    .sum()
    .sort_values(ascending=False)
)

daily_total.head(10)


date
2015-12-24    171169
2016-12-24    167542
2016-12-23    156932
2014-12-24    156546
2013-12-24    155846
2015-12-23    153338
2013-12-23    145876
2014-12-23    144513
2015-12-22    138921
2016-12-22    138892
Name: transactions, dtype: int64

In [24]:
# Top 10 įrašų pagal didžiausią transakcijų skaičių
transactions.sort_values("transactions", ascending=False).head(10)


,date,store_nbr,transactions,weekday,activity_level
52011,2015-12-23,44,8359,Wednesday,High
71010,2016-12-23,44,8307,Friday,High
16570,2013-12-23,44,8256,Monday,High
33700,2014-12-23,44,8120,Tuesday,High
16572,2013-12-23,46,8001,Monday,High
16619,2013-12-24,46,7840,Tuesday,High
16573,2013-12-23,47,7727,Monday,High
52064,2015-12-24,44,7700,Thursday,High
33748,2014-12-24,44,7689,Wednesday,High
70904,2016-12-21,44,7597,Wednesday,High


## 11. Eksportas į Excel

Gera praktika:

- eksportuoti **jau sutvarkytus** ir analizės stulpeliais papildytus duomenis
- `index=False`, kad Excel faile nebūtų techninio indekso stulpelio
- aiškiai įvardyti failą

Žemiau sukuriami du Excel lapai:
- `data` – pilni duomenys
- `summary` – santrauka pagal parduotuvę


In [27]:
# Santrauka pagal parduotuvę
summary_by_store = (
    transactions.groupby("store_nbr")
    .agg(
        days=("date", "nunique"),
        total_transactions=("transactions", "sum"),
        avg_transactions=("transactions", "mean"),
        max_transactions=("transactions", "max"),
    )
    .sort_values("total_transactions", ascending=False)
    .reset_index()
)

summary_by_store.head(10)


,store_nbr,days,total_transactions,avg_transactions,max_transactions
0,44,1677,7273093,4336.966607,8359
1,47,1677,6535810,3897.322600,7727
2,45,1677,6201115,3697.742993,7305
3,46,1677,5990113,3571.921884,8001
4,3,1676,5366350,3201.879475,6085
5,48,1677,5107785,3045.787120,7044
6,8,1676,4637971,2767.285800,5261
7,49,1677,4574103,2727.550984,6600
8,50,1677,4384444,2614.456768,5456
9,11,1676,3972488,2370.219570,5018


In [28]:
# Eksportas į Excel su keliais lapais
output_file = "transactions_analysis.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    transactions.to_excel(writer, sheet_name="data", index=False)
    summary_by_store.to_excel(writer, sheet_name="summary", index=False)

output_file


'transactions_analysis.xlsx'